# Initialisation

In [ ]:
from PyLQR.sim import KDLRobot
from PyLQR.system import PosOrnPlannerSys, PosOrnKeypoint
from PyLQR.solver import BatchILQRCP, BatchILQR, ILQRRecursive
from PyLQR.utils import primitives, PythonCallbackMessage

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # <--- This is important for 3d plotting 
import cv2
import matplotlib.cm as cm
from tqdm import tqdm

import logging
import sys

from scipy.spatial.transform import Rotation 

from contact_grasp.srv import *
from cv_bridge import CvBridge, CvBridgeError

sys.path.append("/home/vdrame/catkin_ws/src/py_panda/PyPanda")
from PyPanda import Robot
import rospy
from PyPanda import Utils

from utils.realsense_utils import RealCamera
from utils.transform_utils import *
from utils.iLQR_wrapper import iLQR
import argparse
from scipy.spatial.transform import Rotation

import time

from sensor_msgs.msg import Image
import rospy

In [ ]:
def run_action(rbt, actions, control_freq, eef_pos=None, eef_quat=None, segmentation_type=None, show_agentview=False, object_range=[5,8]):
    success = False
    rate = rospy.Rate(int(control_freq))

    for idx, action in tqdm(enumerate(actions)):
        rbt.active_controller.send_command(action)
        rate.sleep()
        #env.sim.step()
        if eef_pos is not None:
            eef_pos.append(rbt.model.ee_pos_rel())
        if eef_quat is not None:
            eef_quat.append(rbt.model.ee_orn_rel())
    success = True
    return success, idx, eef_pos, eef_quat

def generate_grasps_client(image, depth):
    rospy.wait_for_service('generate_grasps')
    try:
        generate_grasps = rospy.ServiceProxy('generate_grasps', contactGraspnet)
        resp1 = generate_grasps(image, depth)
        return resp1.quat, resp1.pos, resp1.opening.data, resp1.detected.data
    except rospy.ServiceException as e:
        print("Service call failed: %s"%e)

In [ ]:
# # PATH_TO_URDF = "/home/vdrame/project/client/panda_description/urdf/panda_arm_robosuite.urdf"
# PATH_TO_URDF = "/home/vdrame/Documents/ros_ws/src/rli_ws_base/py_panda/Tutorials/model.urdf"

# BASE_FRAME = "panda_link0"
# TIP_FRAME = "panda_tip"
# # TIP_FRAME = "panda_grasptarget"

# # initial joint config
# q0 = rbt.q
# dq0 = [0]*rbt.dq

# _qMax = np.array([2.87,   1.75,  2.8973, -0.05,  2.8973,  3.75,   2.8973])
# _qMin = np.array([-2.87, -1.75, -2.8973, -3.05, -2.8973, -0.015, -2.8973])
# # self._dqMax = np.array([2., 2., 2., 2., 2.6, 2.6, 2.6])
# _dqMax = np.array([.5, .5, .5, .5, .6, .6, .6])

# rbt_KDL = KDLRobot(PATH_TO_URDF, BASE_FRAME, TIP_FRAME, q0, dq0)
# print(rbt_KDL.get_q())

## Camera init

In [ ]:
# Load camera
camera = RealCamera()
camera.start()

#retrieve image and depth
for i in range(15):
    rgb, depth_image, depth_scale = camera.get_rgb_depth()

rgb, depth_image, depth_scale = camera.get_rgb_depth()
# cv2.imwrite("img.png", rgb)
# cv2.imwrite("depth.png", depth_image)
# cv2.imwrite("depth_rescale.png", depth_image*depth_scale)

depth_scale = 0.001 # depth a publish under millimeter precision
dispose_pos = np.array([0.1, 0.66, 0.1])
dispose_orn_wxyz = np.array([0, 1, 0.35, 0])


# Restart from there

In [ ]:
# #Connect to the robot
rospy.init_node("python_node",anonymous=True)
rbt = Robot("panda",use_gripper=True)

print("rbt end effector pos:", rbt.model.ee_pos_rel())
print("rbt end effector quat (wxyz):", rbt.model.ee_orn_rel())
print("eef rot",  Rotation.from_quat(convert_quat(rbt.model.ee_orn_rel(), to="xyzw")).as_euler("xyz", degrees=True))
print("--------------------------------------------------------------------------")

ee_pose = np.eye(4)
ee_pose[:3,:3] = quat2mat(convert_quat(rbt.model.ee_orn_rel(), to="xyzw")) #xyzw
ee_pose[:3,3] = rbt.model.ee_pos_rel() 

ee2hand = np.eye(4)
ee2hand[2,3] = -0.1034

hand2camera_pos = np.array([0.0488546636437146,-0.03384417860749521,0.0512776975002817]) 
hand2camera_quat = [0.012961267509189803,-0.0012768531849757236,0.7052247395136084,0.708864191484139] #xyzw 
hand2camera_mat = Rotation.from_quat(hand2camera_quat).as_matrix()

hand2camera = np.eye(4)
hand2camera[:3,:3] = hand2camera_mat
hand2camera[:3,3] = hand2camera_pos

camera_pose_world = ee_pose @ ee2hand @ hand2camera

init_pos = rbt.model.ee_pos_rel()
init_orn_wxyz = rbt.model.ee_orn_rel()

# print("camera_pose_world :", camera_pose_world)
# # print("cam pose in world frame:", Rotation.from_matrix(camera_pose_world[:3,:3]).as_euler("xyz", degrees=True))
# print("--------------------------------------------------------------------------")
rbt.gripper.homing()

# Grasping

In [ ]:
eef_pos, eef_quat = [], []

img_cv, depth_image, depth_scale = camera.get_rgb_depth()
depth_cv = depth_image * depth_scale

#Ros CV bridge to convert data from opencv to ROSImage
bridge = CvBridge()

# Call to the server
orn, pos, opening, detected = generate_grasps_client(bridge.cv2_to_imgmsg(img_cv, "bgr8"), bridge.cv2_to_imgmsg(depth_cv, "64FC1"))
# orn, pos = generate_grasps_client(img, depth)

if detected:
    grasps_pos_cam = np.array([pos.x, pos.y, pos.z])
    # grasps_orn_cam_xyzw_raw = np.array([orn.x, orn.y, orn.z, orn.w])
    grasps_orn_cam_xyzw_raw = Rotation.from_quat(np.array([orn.x, orn.y, orn.z, orn.w]))
    rot = np.array([0., 0., 90])
    grasp_rot_tmp = grasps_orn_cam_xyzw_raw.as_euler("XYZ", degrees=True)
    grasps_orn_eul = grasps_orn_cam_xyzw_raw.as_euler("XYZ", degrees=True) + rot
    grasps_orn_cam_xyzw = Rotation.from_euler("XYZ", grasps_orn_eul, degrees=True).as_quat()
    grasps_orn_cam_wxyz = convert_quat(grasps_orn_cam_xyzw, to="wxyz")
    grasps_mat_cam = quat2mat(grasps_orn_cam_xyzw)

    grasp_pos_world = camera_pose_world[:3,:3] @ grasps_pos_cam + camera_pose_world[:3,3]
    grasps_mat_world = camera_pose_world[:3,:3] @ grasps_mat_cam
    grasps_orn_world_xyzw = mat2quat(grasps_mat_world)
    grasp_orn_world_wxyz =  convert_quat(grasps_orn_world_xyzw, to="wxyz")

    print("--------------------------------------------------------------------------")
    print("grasps in camera frame :\n pos :", grasps_pos_cam, "\n grasps_orn_cam :", grasps_orn_cam_xyzw, "\ngrasp world rot",  Rotation.from_quat(grasps_orn_cam_xyzw).as_euler("xyz", degrees=True))
    print("\n\ngrasps in world frame :\n pos :", grasp_pos_world, "\n grasps_orn_world :", grasps_orn_world_xyzw, "\ngrasp world rot",  Rotation.from_quat(grasps_orn_world_xyzw).as_euler("xyz", degrees=True))
    print("--------------------------------------------------------------------------")
else:
    print("No grasping pose detection")
# cv2.imshow('rgb', rgb)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# visualization

In [ ]:
pred_grasps_cam = np.eye(4)
pred_grasps_cam[:3,:3] = grasps_mat_world
pred_grasps_cam[:3,3] = grasp_pos_world
pred_grasps_cam = {1.0: np.array([pred_grasps_cam])}

In [ ]:
plt.switch_backend('QT5Agg')
import mayavi.mlab as mlab

In [ ]:
from utils.visualisation_utils import *
cam_K = camera.getIntrinsic()[0]
z_range = [0.2, 5]
pc_full, pc_segments, pc_colors = extract_point_clouds(depth=depth_cv, K=cam_K, rgb=img_cv, z_range=z_range)

for key in pred_grasps_cam:
    pred_grasps_cam[key][:,:3,3] -= 0.1034 * pred_grasps_cam[key][:,:3,2]

#show_image(self.rgb, self.segmap)
visualize_grasps(pc_full, pred_grasps_cam, plot_opencv_cam=True, pc_colors=pc_colors, add_grasps=None)


# trajectory

In [ ]:
traj_gen = iLQR(rbt)
grasp_jpos, grasp_x_pos, grasp_U, grasp_Ks, grasp_ds = traj_gen.grasping_trajectory(rbt.q, rbt.dq, grasp_pos_world, grasp_orn_world_wxyz, 120)

grasp_q = grasp_jpos[-1]
grasp_dq = np.zeros_like(grasp_q)
dispose_jpos, dispose_x_pos, dispose_U, dispose_Ks, dispose_ds = traj_gen.dispose_trajectory(grasp_q, grasp_dq, grasp_pos_world, grasp_orn_world_wxyz, dispose_pos, dispose_orn_wxyz, 120)
iLQR.plot_trajectory(init_pos, grasp_pos_world, grasp_x_pos, dispose_x_pos)


In [ ]:
rbt.stop_controller()
rbt.error_recovery()
rbt.switch_controller("joint_velocity_controller")
rbt.active_controller


In [ ]:
grasp_U = np.array(grasp_U)
success, idx, eef_pos, eef_quat = run_action(rbt, grasp_U[:80], 20)
rbt.active_controller.send_command(np.zeros(7))
time.sleep(1)
success, idx, eef_pos, eef_quat = run_action(rbt, grasp_U[80:], 20)
rbt.active_controller.send_command(np.zeros(7))
rbt.gripper.move(width=opening-0.015)

# print("rbt end effector pos:", rbt.model.ee_pos_rel())
# print("rbt end effector quat (wxyz):", rbt.model.ee_orn_rel())
# print("eef rot",  Rotation.from_quat(rbt.model.ee_orn_rel()).as_euler("xyz", degrees=True))
# print("--------------------------------------------------------------------------")
# print("\n\ngrasps in world frame :\n pos :", grasp_pos_world, "\n grasps_orn_world :", grasps_orn_world_xyzw, "\ngrasp world rot",  Rotation.from_quat(grasps_orn_world_xyzw).as_euler("xyz", degrees=True))
# print("--------------------------------------------------------------------------")
# print("\n\ILQR final pose in world frame :\n pos :", grasp_x_pos[-1][:3], "\n grasps_orn_world wxyz:", convert_quat(grasp_x_pos[-1][3:], to="xyzw"), "\ngrasp world rot",  Rotation.from_quat(convert_quat(grasp_x_pos[-1][3:], to="xyzw")).as_euler("xyz", degrees=True))

In [ ]:
rbt.gripper.move(width=opening-0.03)


In [ ]:
print(grasp_orn_world_wxyz)
print(rbt.model.ee_orn_world())

In [ ]:
dispose_U = np.array(dispose_U)
success, idx, eef_pos, eef_quat = run_action(rbt, dispose_U, 20)
rbt.active_controller.send_command(np.zeros(7))
rbt.gripper.move(width=0.07)
rbt.gripper.homing()

In [ ]:
return_jpos, return_x_pos, return_U, return_Ks, return_ds = traj_gen.grasping_trajectory(rbt.q, rbt.dq, init_pos, init_orn_wxyz, 120)
return_U = np.array(return_U)
success, idx, eef_pos, eef_quat = run_action(rbt, return_U, 20)
rbt.active_controller.send_command(np.zeros(7))

In [ ]:
print("rbt end effector pos:", rbt.model.ee_pos_rel())
print("rbt end effector quat (wxyz):", rbt.model.ee_orn_rel())
print("eef rot",  Rotation.from_quat(convert_quat(rbt.model.ee_orn_rel(), to="xyzw")).as_euler("xyz", degrees=True))
print("--------------------------------------------------------------------------")
print(init_pos)
print(init_orn_wxyz)

In [ ]:
print("rbt end effector pos:", rbt.model.ee_pos_rel())
print("rbt end effector quat (wxyz):", rbt.model.ee_orn_rel())
print("eef rot",  Rotation.from_quat(convert_quat(rbt.model.ee_orn_rel(), to="xyzw")).as_euler("xyz", degrees=True))
print("--------------------------------------------------------------------------")
print("dispose_pos : ", grasp_pos_world)
print("dispose_orn :", grasp_orn_world_wxyz)

In [ ]:
rbt.stop_controller()

In [ ]:
camera.stop()

In [ ]:
rbt.gripper.grasp(force=10)

In [ ]:
rbt.gripper.move(width=0.07)


In [ ]:
rbt.gripper.homing()

# ros connexion to cam

In [ ]:
# eef_pos, eef_quat = [], []

# # rospy.init_node("python_node",anonymous=True)
# # img_cv, depth_image, depth_scale = camera.get_rgb_depth()
# # depth_cv = depth_image * depth_scale

# #Ros CV bridge to convert data from opencv to ROSImage
# bridge = CvBridge()

# depth = rospy.wait_for_message("/camera/aligned_depth_to_color/image_raw", Image, timeout=2)
# img = rospy.wait_for_message("/camera/color/image_raw", Image, timeout=1)

# img_cv = bridge.imgmsg_to_cv2(img, "bgr8")
# depth_cv = bridge.imgmsg_to_cv2(depth, "64FC1")
# if depth_cv.shape != img_cv.shape[:2]:
#     img_cv = cv2.resize(img_cv, dsize=(depth_cv.shape[1], depth_cv.shape[0]), interpolation=cv2.INTER_AREA)
# depth_cv = depth_cv * 0.001

# # Call to the server
# orn, pos = generate_grasps_client(bridge.cv2_to_imgmsg(img_cv, "bgr8"), bridge.cv2_to_imgmsg(depth_cv, "64FC1"))
# # orn, pos = generate_grasps_client(img, depth)

# grasps_pos_cam = np.array([pos.x, pos.y, pos.z])
# grasps_orn_cam_xyzw = np.array([orn.x, orn.y, orn.z, orn.w])
# grasps_orn_cam_wxyz = np.array([orn.w, orn.x, orn.y, orn.z])
# grasps_mat_cam = quat2mat(grasps_orn_cam_xyzw)

# grasp_pos_world = camera_pose_world[:3,:3] @ grasps_pos_cam + camera_pose_world[:3,3]
# grasps_mat_world = camera_pose_world[:3,:3] @ grasps_mat_cam
# grasps_orn_world_xyzw = mat2quat(grasps_mat_world)
# grasp_orn_world_wxyz =  convert_quat(grasps_orn_world_xyzw, to="wxyz")

# print("--------------------------------------------------------------------------")
# print("grasps in camera frame :\n pos :", grasps_pos_cam, "\n grasps_orn_cam :", grasps_orn_cam_xyzw, "\ngrasp world rot",  Rotation.from_quat(grasps_orn_cam_xyzw).as_euler("xyz", degrees=True))
# print("\n\ngrasps in world frame :\n pos :", grasp_pos_world, "\n grasps_orn_world :", grasps_orn_world_xyzw, "\ngrasp world rot",  Rotation.from_quat(grasps_orn_world_xyzw).as_euler("xyz", degrees=True))
# print("--------------------------------------------------------------------------")

# # cv2.imshow('rgb', rgb)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()